# PyTorch semantic segmentation 
For this last exercise you are going to work on a new computer vision task called semantic segmentation where not an entire image but each pixel is classified. This means in comparison to image classification your model outputs not a single class but a segmentation with the same size as the input image. Since you have already learned a lot in the past exercises and you are now familiar with the basics of PyTorch the model design and architecture as well as the training is totally up to you. We only provide you with the train and validation dataset and recommend you to look for PyTorch implementations and finetuning weights of famous semantic segmentation networks.

<img src="https://camo.githubusercontent.com/d10b897e15344334e449104a824aff6c29125dc2/687474703a2f2f63616c76696e2e696e662e65642e61632e756b2f77702d636f6e74656e742f75706c6f6164732f646174612f636f636f7374756666646174617365742f636f636f73747566662d6578616d706c65732e706e67">

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.autograd import Variable

from dl4cv.classifiers.segmentation_network import SegmentationNetwork
from dl4cv.data_utils import SegmentationData

#torch.set_default_tensor_type('torch.FloatTensor')

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

## Train and validation data

Download the dataset with the `datasets/get_segmentation_dataset.sh` script and make yourself familiar with it. Look at the labels described in `datasets/segmentation_data/info.html` and note the label `unlabeled`. Pixels with the label `unlabeled` should neither be considered in your loss nor the accuracy.

In [ ]:
train_data = SegmentationData(root='datasets/segmentation_data',
                              image_list='train.txt')

val_data = SegmentationData(root='datasets/segmentation_data',
                            image_list='val.txt')

In [ ]:
print("Train size: %i" % len(train_data))
print("Validation size: %i" % len(val_data))

img, target = train_data[1]
print("Img size: " + str(img.size()))
print("Segmentation size: " + str(target.size()))
plt.figure(figsize=(4,4))
plt.imshow(img.numpy().transpose(1,2,0))
plt.show()
plt.figure(figsize=(4,4))
plt.imshow(target.numpy())
plt.show()

## Train your network
Implement your network architecture in `classifiers.segmentation_network.py` and adapt your `Solver` to process segmentation labels. Think about finetuning a model instead of training from scratch. Maybe even train a model on the *GPU* using your own or our computer lab *GPUs*. See our `README.md` on how to use *Jupyter* notebooks remotely.

In [ ]:
from dl4cv.classifiers.segmentation_network import SegmentationNetwork
from dl4cv.classifiers.solver import Solver

############################################################################
#                             YOUR CODE                                    #
############################################################################

# Test your model
Since this dataset is very small we only provided you with a validation set.

In [ ]:
test_data = SegmentationData(root='datasets/segmentation_data', image_list='val.txt')
test_loader = torch.utils.data.DataLoader(test_data,
                                          batch_size=1,
                                          shuffle=False,
                                          num_workers=1)

test_scores = []
model.eval()
for batch in test_loader:
    inputs, labels = Variable(batch[0]), Variable(batch[1])

    outputs = model.forward(inputs)
    _, preds = torch.max(outputs, 1)
    labels_mask = labels >= 0
    test_scores.append(np.mean((preds == labels)[labels_mask].data.numpy()))
    
model.train()
test_acc = np.mean(test_scores)
print(test_acc)

# Visualization

In [ ]:
from dl4cv.data_utils import label_img_to_rgb

img, target, pred = inputs[0].data.numpy(), labels[0].data.numpy(), preds[0].data.numpy()
              
plt.figure(figsize=(4,4))
plt.imshow(img.transpose(1,2,0))
plt.show()
plt.figure(figsize=(4,4))
plt.imshow(label_img_to_rgb(target))
plt.show()
plt.figure(figsize=(4,4))
plt.imshow(label_img_to_rgb(pred))
plt.show()

## Save the model

When you are satisfied with your training, save the model for submission.

In [ ]:
model.save("models/segmentation_network.model")

## Scoring function
We will score the model you have just saved based on the classification accuracy on our test dataset. The scoring function should represent the difficulty of obtaining a good test accuracy and should therefore give 0 points for worse results than random guessing, should be linear in a first regime and exponential beyond that. The onset of exponential growth depends on the problem. In that region you get twice as many points for an additional 10% accuracy.

For this problem we specifically use the following scoring function:
    $$f(x) = \left\{
	\begin{array}{ll}
		0  & \mbox{if } x \leq 0.1 \\
		100x & \mbox{if } 0.1 < x \leq 0.5 \\
        \left(\frac{50}{\exp(0.5 \ln(2)/0.1)}\right) \exp(x \ln(2)/0.1) & \mbox{if } 0.5 < x \leq 1
	\end{array}
\right.$$

The function can be plotted in the following cell.

In [ ]:
from dl4cv.data_utils import scoring_function

x = np.linspace(0, 1, num=1000)
plt.plot(x, scoring_function(x, lin_exp_boundary=0.5, doubling_rate=0.1))
plt.title('Scoring Function')
plt.xlabel('Test Accuracy')
plt.ylabel('Score')
plt.show()